<a href="https://colab.research.google.com/github/ramadhanaraz/distilBERT-bangkit/blob/main/TF_Serving_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TensorFlow and tf.keras
print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.26.0

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

Installing dependencies for Colab environment
     |████████████████████████████████| 2.4 MB 5.0 MB/s 
TensorFlow version: 2.8.2


In [ ]:
!mkdir /model
!cp -R drive/MyDrive/distilBERT /model/distilBERT

mkdir: cannot create directory ‘/model’: File exists


In [ ]:
!saved_model_cli show --dir /model/distilBERT/IOH_dBERT --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['Desc_input'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 64)
        name: serving_default_Desc_input:0
    inputs['Desc_mask'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 64)
        name: serving_default_Desc_mask:0
    inputs['Title_input'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 64)
        name: serving_default_Title_input:0
    inputs['Title_mask'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 64)
        name: serving_defau

In [ ]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   143k      0 --:--:-- --:--:-- --:--:--  143k
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 http://ppa.launchpad.n

In [ ]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tensorflow-model-server is already the newest version (2.9.0).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 87 not upgraded.


In [ ]:
import os
os.environ["MODEL_DIR"] = "/model/distilBERT/IOH_dBERT"

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=dBERT_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


Starting job # 0 in a separate thread.


In [ ]:
!tail server.log

tensorflow_model_server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.28' not found (required by tensorflow_model_server)
tensorflow_model_server: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.26' not found (required by tensorflow_model_server)
tensorflow_model_server: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by tensorflow_model_server)


In [ ]:
!pip install transformers
from transformers import TFDistilBertModel, DistilBertTokenizer, DistilBertConfig

## DistilBERT load
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def toEncoding(sentences, max_len):  
  input_ids=[]
  attention_masks=[]

  for sentence in sentences:
      dbert_inputs = dbert_tokenizer.encode_plus(sentence, 
                                               add_special_tokens = True, 
                                               max_length = max_len, 
                                               pad_to_max_length = True, 
                                               return_attention_mask = True, 
                                               truncation = True)
      input_ids.append(dbert_inputs['input_ids'])
      attention_masks.append(dbert_inputs['attention_mask'])

  input_ids = np.asarray(input_ids)
  attention_masks = np.array(attention_masks)

  return input_ids, attention_masks

In [ ]:
max_len = 64
test_title = "Fix Website"
test_description = "Web has been overloading for the past days. It might be due to the backend not being optimized"

In [ ]:
title_input, title_mask = toEncoding(test_title, max_len)
desc_input, desc_mask = toEncoding(test_description, max_len)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
title_input, title_mask = title_input.tolist(), title_mask.tolist()
desc_input, desc_mask = desc_input.tolist(), desc_mask.tolist()

In [ ]:
import json
json_dict = {"signature_name": "serving_default", "instances": [[desc_input, title_input], [desc_mask, title_mask]]}
data = json.dumps(json_dict)
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ...  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]]}


In [ ]:
with open("body.json", "w") as outfile:
    json.dump(json_dict, outfile)

In [ ]:
import requests

headers = {"content-type": "application/json"}
json_response = requests.post('http://1676-35-237-46-23.ngrok.io/v1/models/dBERT_model:predict', data=data, headers=headers)
print(json_response.text)
predictions = json.loads(json_response.text)['predictions']

t=2022-06-12T06:18:09+0000 lvl=warn msg="failed to open private leg" id=c4b69c17b7ac privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"



<!doctype html5>
<html>
    <head>
        <style type="text/css">
         
        strong { font-weight: bold; }
        hr { -moz-box-sizing: content-box; box-sizing: content-box; height: 0; }
        html { font-family: sans-serif;   -ms-text-size-adjust: 100%;   -webkit-text-size-adjust: 100%;   } body { margin: 0; }
        a { background-color: transparent; }
        a:active, a:hover { outline: 0; }
        </style>

        <style type="text/css">
            body { background-color: #f5f5f5; }
            .container { width: 500px; margin: auto; color: #444; padding: 5px; }
            a, strong { color: purple; text-decoration: none; }
            a:hover { text-decoration: underline; }
            h2 { text-align: center; color: #000; }
            p { line-height: 20px; }
        </style>
    </head>
    <body>
        <div class="container">
            
            
<h2>Failed to complete tunnel connection</h2>
<hr />
<p>
    The connection to <strong><a href="http://16

JSONDecodeError: ignored

In [ ]:
!pip install pyngrok

from pyngrok import ngrok
model_tunnel = ngrok.connect(8501)
print(model_tunnel)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


t=2022-06-12T06:14:12+0000 lvl=warn msg="failed to start tunnel" pg=/api/tunnels id=2289da00a4db2291 err="Your account may not run more than 2 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2ASqSFp8B7WAySv7ClmQymBXJI6, tn_2ASqSBM8eO7kCwTKYzlEvRff6BG\n\r\n\r\nERR_NGROK_324\r\n"


PyngrokNgrokHTTPError: ignored